# How to add your own transforms in the ClinicaDL Transforms Zoo ?


The goal of this tutorial is to add your own transforms to the ClinicaDL Transform Zoo because you think it can be usefull for other people. 

## STEP 1: Create the Transform

First of all, create a new python file in `clinicadl_zoo/transforms/zoo` in which you create a new class that inherits from [torchio.Transforms](https://torchio.readthedocs.io/transforms/transforms.html#torchio.transforms.Transform). 

You only need to overwrite the ``__init__``, where you define your parameters (don't forget to define the list of these parameters is ``self.arg_names``), and the method ``apply_transform``, where you actually compute the transform.

This latter must take as input and return a [DataPoint](https://clinicadl.readthedocs.io/en/latest/data/datapoint.html#clinicadl.data.structures.DataPoint). Inside, feel free to use the useful methods of DataPoint to compute your transforms. You will likely be particularly interested in ``get_images``, that enables you to iterate on all the images and masks in the DataPoint.

PS: don't forget to call ``super().__init__()``!

In [ ]:
# in zoo/my_custom_transform.py 

import torchio as tio
from clinicadl.data.structures import DataPoint

class MyCustomTransform(tio.Transform):
    """
    To create a Custom transform in ClinicaDL. 
    Don't forget to write the docstring.
    """

    def __init__(self, arg1: float, arg2: int, **kwargs):
        """
        Don't forget to define the type of the arguments.
        Args:
            arg1 (int): Description of arg1.
            arg2 (str): Description of arg2.
            **kwargs: Additional arguments to be passed to the parent class.
        """
        super().__init__(**kwargs)
        self.arg1 = arg1
        self.arg2 = arg2
        self.args_names = ["arg1", "arg2"]

    def apply_transform(self, datapoint: DataPoint) -> DataPoint:
        
        transformed_datapoint = datapoint
        return transformed_datapoint

## STEP 2: Add the name of your transform

First you need to add name of your transform in the ``ZooTransform`` enum in the `clinicadl_zoo.transforms.enum`.

In [ ]:
#in zoo/config/enum.py

from clinicadl.utils.enum import BaseEnum


class ZooTransform(str, BaseEnum):
    """
    Implemented transforms in ClinicaDL Transofrm Zoo.
    see: https://torchio.readthedocs.io/transforms/transforms.html
    """

    NAN_REMOVAL = "NanRemoval"
    MY_CUSTOM_TRANSFORM = "MyCustomTransform"


## STEP 3: Create the config class asociated to your transform

ClinicaDL uses configuration classes to enhance the reproducibility of your code.  
For each transform compatible with ClinicaDL, a corresponding config class is required.  
This class must inherit from `TransformConfig` in `clinicadl.transforms.config.base` and **must be named** as the transform followed by "Config".

To implement your config class, you only need to override the following:

- The ``__init__`` method, where you define your parameters (as shown above),
- The ``name`` property, which should match the value you added in the ``ZooTransform`` enum,
- The ``_get_class`` method, which should return the transform class associated with this configuration (i.e., the one you just implemented above).


In [ ]:
# in zoo/config/my_custom_transform_config.py
from typing import Union

import torchio as tio
from pydantic import computed_field


from clinicadl.utils.config import DefaultFromLibrary
from clinicadl.transforms.config import TransformConfig

# from clinicadl_zoo.transforms import ZooTransform
# from ..transforms import MyCustomTransform


class MyCustomTransformConfig(TransformConfig):
    """
    Config class for :py:class:`clinicadl.transforms.zoo.transforms.MyCustomTransform <clinicadl.transforms.zoo.transforms.MyCustomTransform>`.
    """

    arg1: float
    arg2: int

    def __init__(
        self,
        arg1: Union[float, DefaultFromLibrary] = DefaultFromLibrary.YES,
        arg2: Union[int, DefaultFromLibrary] = DefaultFromLibrary.YES,
    ):
        super().__init__(arg1=arg1, arg2=arg2)

    @computed_field
    @property
    def name(self) -> str:
        """The name of the transform."""
        return ZooTransform.MY_CUSTOM_TRANSFORM.value 

    def _get_class(self) -> type[tio.Transform]:
        """Returns the transform associated to this config class."""
        return MyCustomTransform


> 💡 **Reminder:** Don’t forget to update the relevant `__init__.py` files and the `zoo.config.factory.py` with the necessary imports if you want your transform to be accessible throughout the codebase.


Now you should be able to use the `CapsDataset` module with your custom transform just like any other transform in ClinicaDL.  
If needed, you can refer to our [tutorial on how to use the `CapsDataset` module](#) for guidance.


## STEP 4: Create a Pull Request in the repository 

Once everything is implemented and tested, you should open a pull request on the [ClinicaDL GitHub repository](https://github.com/aramis-lab/clinicadl).  
Make sure to briefly describe your transform, its purpose, and any parameters it introduces. This helps the maintainers review and integrate your contribution more efficiently.

If you have any questions or run into issues during the process, don’t hesitate to contact us — we’re happy to help!
